In [1]:
import sys
import numpy as np
from tqdm import tqdm_notebook
import pickle
import pandas as pd
import plotly
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import time
import init as z
import simulate_corona as sc
import simulate_corona_senior as scs
import multiprocessing as mp
import load_part as lp

sns.set(rc={'figure.figsize':(11, 4)})

## definitions
out_filename_pct0 = "./out/simul_SIR_pct"
out_filename_raw0 = "./out/simul_SIR_raw"
out_figname = "./fig/"
res_rootname = './out/'
patname = "simul_SIR_"

def load_data(suf):
    ax = lp.load_file_pickle(res_rootname,patname+"_raw_"+suf)
    with open(out_filename0+"_"+suf+".pickle",'rb') as fid:
        bx = pickle.load(fid)
    fid.close()
    
    return a,b
    
## load data
pct_SIR_high,SIR_high = load_data("high")
pct_SIR_med,SIR_med = load_data("med")
pct_SIR_low,SIR_low = load_data("low")
pct_SIR_sen,SIR_sen = load_data("low_senior")

## 1. compare peaks (as a function of pop.mobility alpha), relative numbers
def plot_peaks(data_arr,colname):
    for data in (data_arr):
        plt.plot(x,data.inf[0:z.simul_len],c=colname,alpha=.9)
        plt.xlim((0,z.simul_len))
        plt.ylim((0,.5))

x = np.arrange(1,z.simul_len+1)
plt.rcParams['axes.facecolor']='white'
plot_peaks(pct_SIR_high,'red')
plot_peaks(pct_SIR_med,'orange')
plot_peaks(pct_SIR_low,'green')
plot_peaks(pct_SIR_sen,'blue')
plt.title("Porovnanie peaku infekcie podla mobility populacie")
plt.xlabel("Dni")
plt.ylabel("Pomer nakazenych")
plt.savefig(out_figname+"main_plus_senior.png",dpi=300)
plt.close

2
62


In [ ]:
## initialize
np.random.seed(123)
percSIR_high=[]
percSIR_med=[]
percSIR_low=[]
percSIR_low_senior=[]
SIR_high=[]
SIR_med=[]
SIR_low=[]
SIR_low_senior=[]

## setting
run_multicore = True
run_type = 1 # 1=high 2=med 3=low 4=senior

alpha_vec_sen = np.full(x.simul_cnt, x.public_trans_low)
alpha_vec_low = np.full(x.simul_cnt, x.public_trans_low)
alpha_vec_mid = np.full(x.simul_cnt, x.public_trans_mid)
alpha_vec_high = np.full(x.simul_cnt, x.public_trans_high)

def run_simulation(alpha_vec,fun):
    ra = []
    rb = []
    if run_multicore:
        cores_cnt = mp.cpu_count()
        pool = mp.Pool(cores_cnt)
        tic = time.perf_counter()
        res = pool.map(fun,alpha_vec)
        toc = time.perf_counter()
        pool.close()
        dt_avg = (toc-tic)/x.simul_cnt
        k=0
        for sim in np.arange(x.simul_cnt):
            ra0,rb0 = res[k]
            ra.append(ra0)
            rb.append(rb0)
            k +=1
    else:
        k=0
        tic = time.perf_counter()
        for sim in np.arange(x.simul_cnt):    
            print(k)
            ra0,rb0 = sc.simul(alpha_vec[k])
            ra.append(ra0)
            ra.append(rb0)
            k +=1
        toc = time.perf_counter()
        dt_avg = (toc-tic)/x.simul_cnt
    return ra,rb,dt_avg      
    

## run all simulations and  save results
percSIR_high,SIR_high,dt_avg_high=run_simulation(alpha_vec_high,sc.simul)
print(dt_avg_high)
with open('./out/simul_percSIR_high.pickle','wb') as f:
    pickle.dump([percSIR_high],f)
sp.save_file_pickle(SIR_high,'./out/simul_SIR_high')

#percSIR_med,SIR_med,dt_avg_med=run_simulation(alpha_vec_mid,sc.simul)
#print(dt_avg_med)
#with open('./out/simulations_med.pickle','wb') as f:
#    pickle.dump([percSIR_med,SIR_med],f)

#percSIR_low,SIR_low=run_simulation(alpha_vec_low,sc.simul)
#with open('./out/simulations_low.pickle','wb') as f:
#    pickle.dump([percSIR_low,SIR_low],f)

#percSIR_low_senior,SIR_low_senior=run_simulation(alpha_vec_sen,scs.simul)
#with open('./out/simulations_low_senior.pickle','wb') as f:
#    pickle.dump([percSIR_low_senior,SIR_low_senior],f)
f.close()

In [ ]:
import save_part as sp

sp.save_file_pickle(SIR_high,'./out/ssim_high')

In [ ]:
y=SIR_high
n = len(y)
filenameroot = './out/sim_high'
f = open(filenameroot+'.pickle','wb')
k=0
nn = n-1
nn0 = nn
nn1 = 0
cont = True
while cont:
    print([k, nn])
    yy = y[0:nn]
    try:
        # yy = y[0:nn]
        pickle.dump(yy,f)
        excthrown = False
    except:
        excthrown = True
    print(excthrown)
    if excthrown:
        nn0 = nn
        if nn<=1:
            cont = False
        else:
            if nn1>0:
                nn = int((nn+nn1)/2)
            else:
                nn = int((nn+1)/2)-1
    else:
        if nn1 == nn or nn>=n-2:
            cont = False
        else:        
            nn1 = nn
            nn = int((nn+nn0)/2)
    k+=1  
            

In [ ]:
import save_safe as s

In [ ]:
print(nn)
fsize = nn+1
filecnt = 1+int(n/(nn+1))
print(filecnt)
k=0
for fidx in range(filecnt):
    filename0 = filenameroot+'_'+str(k)+'.pickle'
    f0 = open(filename0,'wb')
    ei = min((k+1)*fsize,n)
    yy = y[k*fsize:ei-1]
    pickle.dump(yy,f0)
    k+=1
    f0.close()
    

In [ ]:
print(len())